In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# for scientific and statistical computing
import scipy as sp
#To make working with geospatial data in python easier
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
plt.style.use('ggplot')

In [ ]:
df=pd.read_csv("/kaggle/input/electric-vehicle-population-data/Electric_Vehicle_Population_Data.csv")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.columns.unique()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending = False)

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe().style.background_gradient(cmap='Blues')

In [ ]:
plt.rcParams['figure.figsize'] = (15,6)
sns.heatmap(df.isnull(),yticklabels = False, cbar = False , cmap = 'viridis')
plt.title("Missing null values")

In [ ]:
# #### Get a clearer picture of missing data with this nifty code snippet! See the percentage of null values in our dataset sorted in ascending order,
# making it easy to identify which features have the most missing data.
missing_percentage = (df.isnull().sum().sort_values(ascending = False)/len(df))*100
missing_percentage

In [ ]:
df['Electric Utility'] = df['Electric Utility'].fillna('Utility Not Avalilable')
df['Legislative District'] = df['Legislative District'].fillna('Unknown')
df['Vehicle Location'] = df['Vehicle Location'].fillna('Unknown')
df['Model'] = df['Model'].fillna('Unknown')
df['2020 Census Tract'] = df['2020 Census Tract'].fillna('Unknown')
df['City'] = df['City'].fillna('Unknown')
# fill null values in the 'Postal Code' column with the mean salary
mean_postcode = df['Postal Code'].mean()
df['Postal Code'] = df['Postal Code'].fillna(mean_postcode)
df['County'] = df['County'].fillna('Unknown')

In [ ]:
#### Get a clearer picture of missing data with this nifty code snippet! See the percentage of null values in our dataset sorted in ascending order,
# making it easy to identify which features have the most missing data.
missing_percentage1 = (df.isnull().sum().sort_values(ascending = False)/len(df))*100
missing_percentage1

In [ ]:
# convert the 'Postal Code ' attribute from float to integer
df['Postal Code'] = df['Postal Code'].astype(int)

In [ ]:
df.info()

In [ ]:
# rename columns
df = df.rename(columns={'Postal Code': 'PostalCode', 'Model Year': 'ModelYear','Electric Vehicle Type': 'Electric_Vehicle_Type',
'Clean Alternative Fuel Vehicle (CAFV) Eligibility':'CAFV_Eligibility','Electric Range':'Electric_Range', 'Base MSRP':'Base_MSRP', 'Legislative District':'Legislative_District',
'DOL Vehicle ID': 'DOL_Vehicle_ID','Vehicle Location':'Vehicle_Location','Electric Utility':'Electric_Utility','2020 Census Tract':'2020_Census_Tract','VIN (1-10)':'VIN'})
print(df.columns)

In [ ]:
##have a view of refreshed dataset again
df.isnull().sum().sort_values(ascending = False)

In [ ]:
df.duplicated().any()

In [ ]:
df.columns.unique()

In [ ]:
any_duplicates = df.duplicated(subset=['VIN', 'County', 'City', 'State','PostalCode', 'ModelYear', 'Make',
'Model', 'Electric_Vehicle_Type', 'CAFV_Eligibility', 'Electric_Range',
'Base_MSRP', 'Legislative_District', 'DOL_Vehicle_ID',
'Vehicle_Location', 'Electric_Utility', '2020_Census_Tract']).any()

In [ ]:
any_duplicates

In [ ]:
df.describe().style.background_gradient(cmap='Blues')

In [ ]:
#checking the corrlation of all the attributes with target
plt.figure(figsize=(15, 6))
cmap = "cividis"
heatmap=sns.heatmap(df.corr(),cmap=cmap, annot=True)
heatmap.set_title('Electric Vehicle Correlation Heatmap', fontdict={'fontsize':12}, pad=12)

In [ ]:
# Create a histogram of the PostalCode variable
plt.hist(df["ModelYear"],bins=10,edgecolor='blue')
plt.xlabel("ModelYear")
plt.ylabel("Count")

In [ ]:
plt.hist(df["Electric_Range"],bins=10,edgecolor='blue')
plt.xlabel("Electric_Range")
plt.ylabel("Count")

In [ ]:
plt.hist(df["Base_MSRP"],bins=20,edgecolor='blue')
plt.xlabel("Base_MSRP")
plt.ylabel("Count")

In [ ]:
plt.hist(df["DOL_Vehicle_ID"],bins=20,edgecolor='blue')
plt.xlabel("DOL_Vehicle_ID")
plt.ylabel("Count")

In [ ]:
df['Base_MSRP'].unique()

In [ ]:
df.drop('Base_MSRP', axis=1, inplace=True)
df.info()

In [ ]:
df['Electric_Utility'].unique()

In [ ]:
def categorize_electric_utility(value):
    if value == "Utility Not Avalilable":
        return "Not Available"
    elif "||" in value:
        return "Multi type Utilities"
    elif "|" in value:
        return "Single Type Utilities"
    else:
        return "Only One Option"

df['Electric_Utility_Category'] = df['Electric_Utility'].apply(categorize_electric_utility)

In [ ]:
df['Electric_Utility_Category'].unique()

In [ ]:
df.head()

In [ ]:
df['Legislative_District'].unique()

In [ ]:
df.drop('Legislative_District', axis=1, inplace=True)
df.info()

**We are going to pose following questions against the dataset:**
 1. What are the Top 10 count of cars per county?
 2. What are the Top 10 count of cars per city?
 3. What are the Top 10 count of cars per state?
 4. What are the Top 10 count of cars per postal code?
 5. What are the Top 10 Make distribution count per top 10 County?
 6. What are the Top 10 Make distribution count per top 10 City?
 7. What are the Top 10 Make distribution count per top 10 State?
 8. What are the EVT Distribution count per state?
 9. What are the Electric Utility Distribution in top 10 cities with highest number of cars?
 10. What are the Top 5 vs Bottom 5 Comparison?
 11. What are the Distances Travel by vehicle make per electric charge?
 12. What are the Year Wise Cars sales growth?
 13. Plotting the lat and lon

**Question 1. What are the Top 10 count of cars per county?**

In [ ]:
car_counts = df['County'].value_counts().nlargest(10)
# plot the counts
car_counts.plot(kind='bar')
plt.xlabel('County')
plt.ylabel('Number of Cars')
plt.title('Top 10 County of Cars per County')
plt.show()
car_counts_df = car_counts.to_frame()
car_counts_df.style.background_gradient(cmap='Blues')

**Question 2. What are the Top 10 count of cars per city?**

In [ ]:
car_counts_Cty = df['City'].value_counts().nlargest(10)
# plot the counts
car_counts_Cty.plot(kind='bar')
plt.xlabel('City')
plt.ylabel('Number of Cars')
plt.title('Top 10 Count of Cars per City')
plt.show()
car_counts_cty_df = car_counts_Cty.to_frame()
car_counts_cty_df.style.background_gradient(cmap='Blues')

**Question 3. What are the Top 10 count of cars per state?**

In [ ]:
car_counts_St = df['State'].value_counts().nlargest(10)
# plot the counts with a logarithmic y-axis
plt.bar(car_counts_St.index, car_counts_St.values)
plt.yscale('log')
plt.xlabel('State')
plt.ylabel('Number of Cars (log scale)')
plt.title('Top 10 Count of Cars per State')
plt.show()
car_counts_St_df = car_counts_St.to_frame()
car_counts_St_df.style.background_gradient(cmap='Blues')

**Question 4. What are the Top 10 count of cars per postal code?**

In [ ]:

car_counts_pc = df['PostalCode'].value_counts().nlargest(10)
# plot the counts with a logarithmic y-axis
plt.bar(car_counts_pc.index, car_counts_pc.values)
#plt.yscale('log')
plt.xlabel('Postal Code')
plt.ylabel('Number of Cars')
plt.title('Top 10 Count of Cars per Postal Code')
plt.show()
car_counts_pc_df = car_counts_pc.to_frame()
car_counts_pc_df.style.background_gradient(cmap='Blues')

**Question 5. What are the Top 10 Make distribution count per top 10 County?**

In [ ]:
cnt_MkCounty = df.groupby(['County', 'Make']).size().reset_index(name='Count')
# Group the data by county and make, and sum the counts for each group
grouped_data = cnt_MkCounty.groupby(['County', 'Make'])['Count'].sum().reset_index()
# Group the data by county and sum the counts for each county
county_counts = grouped_data.groupby('County')['Count'].sum().reset_index()
make_counts = grouped_data.groupby('Make')['Count'].sum().reset_index()
# Sort the counties by count in descending order, and select the top 10
top_counties = county_counts.sort_values(by='Count', ascending=False).head(10)
top_makes = make_counts.sort_values(by='Count', ascending=False).head(10)
# Filter the data to only include the top 10 counties
filtered_data = grouped_data[grouped_data['County'].isin(top_counties['County']) & grouped_data['Make'].isin(top_makes['Make'])]
# Pivot the data to create a matrix with counties as rows, makes as columns,and counts as values
pivoted_data = filtered_data.pivot(index='County', columns='Make',values='Count').fillna(0)
# Create a stacked bar plot of the pivoted data
pivoted_data.plot(kind='bar', stacked=True)
# Set the title and axis labels
plt.title('Top 10 Make distribution count per top 10 County')
plt.xlabel('County')
plt.ylabel('Number of Cars')
# Show the plot
plt.show()
pivoted_data.head()

**Question 6 What are the Top 10 Make distribution count per top 10 City?**

In [ ]:
cnt_MkCity = df.groupby(['City', 'Make']).size().reset_index(name='Count')
# Group the data by county and make, and sum the counts for each group
grouped_data_cty = cnt_MkCity.groupby(['City', 'Make'])['Count'].sum().reset_index()
# Group the data by county and sum the counts for each county
city_counts = grouped_data_cty.groupby('City')['Count'].sum().reset_index()
make_counts = grouped_data_cty.groupby('Make')['Count'].sum().reset_index()
# Sort the counties by count in descending order, and select the top 10
top_cities = city_counts.sort_values(by='Count', ascending=False).head(10)
top_makes = make_counts.sort_values(by='Count', ascending=False).head(10)
# Filter the data to only include the top 10 counties
filtered_data_Cty = grouped_data_cty[grouped_data_cty['City'].isin(top_cities['City']) & grouped_data_cty['Make'].isin(top_makes['Make'])]
# Pivot the data to create a matrix with counties as rows, makes as columns, and counts as values
pivoted_data_cty = filtered_data_Cty.pivot(index='City', columns='Make',values='Count').fillna(0)
# Create a stacked bar plot of the pivoted data
pivoted_data_cty.plot(kind='bar', stacked=True)
# Set the title and axis labels
plt.title('Top 10 Make distribution count per top 10 City')
plt.xlabel('City')
plt.ylabel('Number of Cars')
# Show the plot
plt.show()
pivoted_data_cty.head()

****

**Question 7 What are the Top 10 Make distribution count per top 10 State?**

In [ ]:
cnt_Mk_St = df.groupby(['State', 'Make']).size().reset_index(name='Count')
# Group the data by county and make, and sum the counts for each group
grouped_data_St = cnt_Mk_St.groupby(['State', 'Make'])['Count'].sum().reset_index()
# Group the data by county and sum the counts for each county
st_counts = grouped_data_St.groupby('State')['Count'].sum().reset_index()
make_counts = grouped_data_St.groupby('Make')['Count'].sum().reset_index()
# Sort the counties by count in descending order, and select the top 10
top_States = st_counts.sort_values(by='Count', ascending=False).head(10)
top_makes = make_counts.sort_values(by='Count', ascending=False).head(10)
# Filter the data to only include the top 10 counties
filtered_data_St = grouped_data_St[grouped_data_St['State'].isin(top_States['State']) & grouped_data_St['Make'].isin(top_makes['Make'])]
# Pivot the data to create a matrix with counties as rows, makes as columns,and counts as values
pivoted_data_St = filtered_data_St.pivot(index='State', columns='Make',values='Count').fillna(0)
# Create a stacked bar plot of the pivoted data
pivoted_data_St.plot(kind='bar', stacked=True)
# Set the title and axis labels
plt.title('Top 10 Make distribution count per top 10 State')
plt.yscale('log')
plt.xlabel('State')
plt.ylabel('Number of Cars')
# Show the plot
plt.show()
pivoted_data_St.head(10)

**Question 8. What are the EVT Distribution count per state?**

In [ ]:
## EVT Distribution count per state
evt_Dis_cnt = df.groupby(['State', 'Electric_Vehicle_Type']).size().reset_index(name='Count')
evt_Dis_cnt.groupby(['State', 'Electric_Vehicle_Type'])['Count'].count().unstack().plot(kind='bar', stacked=True)
# Move legend outside of plot
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')
plt.show()
evt_Dis_cnt.head(10).sort_values(by='Count', ascending=False)

**Question 9. What are the Electric Utility Distribution in top 10 cities with highest number of cars?**

In [ ]:
cnt_Cty_EUD = df.groupby(['City', 'Electric_Utility_Category']).size().reset_index(name='Count')
# Group the data by county and make, and sum the counts for each group
grouped_data_Cty_EUD = cnt_Cty_EUD.groupby(['City','Electric_Utility_Category'])['Count'].sum().reset_index()
# Group the data by county and sum the counts for each county
Cty_EUD_counts = grouped_data_Cty_EUD.groupby('City')['Count'].sum().reset_index()
# make_counts = grouped_data_St.groupby('Make')['Count'].sum().reset_index()
# Sort the counties by count in descending order, and select the top 10
top_Cty_EUD_counts = Cty_EUD_counts.sort_values(by='Count', ascending=False).head(30)
# top_makes = make_counts.sort_values(by='Count', ascending=False).head(10)
# Filter the data to only include the top 10 counties
filtered_data_Cty_EUD = grouped_data_Cty_EUD[grouped_data_Cty_EUD['City'].isin(top_Cty_EUD_counts['City'])]
# Pivot the data to create a matrix with counties as rows, makes as columns, and counts as values
pivoted_data_Cty_EUD = filtered_data_Cty_EUD.pivot(index='City',columns='Electric_Utility_Category', values='Count').fillna(0)
# Create a stacked bar plot of the pivoted data
pivoted_data_Cty_EUD.plot(kind='bar', stacked=True)
# Set the title and axis labels
plt.title('Electric Utility Distribution in top 10 cities with highest number of cars')
# plt.yscale('log')
plt.xlabel('City')
plt.ylabel('Number of EUC')
# Show the plot
plt.show()
pivoted_data_Cty_EUD.head(10)

**Question 10. What are the Top 5 vs Bottom 5 Comparison?**

In [ ]:
car_market_share = pd.DataFrame(df.groupby('Make')['VIN'].count().reset_index()).sort_values(by='VIN',ascending=False).reset_index(drop=True)
car_market_share.columns = ['model','count']
car_market_share_top_5 = car_market_share.head(5)
car_market_share_bottom_5 = car_market_share.tail(5)
fig, axes = plt.subplots(1, 2, figsize=(20, 5))
sns.barplot(data=car_market_share_top_5, x="model", y="count",ax=axes[0])
sns.barplot(data=car_market_share_bottom_5, x="model", y="count",ax=axes[1])
axes[0].set_title('Top 5 Cars')
axes[1].set_title('Bottom 5 Cars')

**Question 11. What are the Distances Travel by vehicle make per electric charge?**

In [ ]:
import plotly.express as ps
km_range = pd.DataFrame(df.groupby('Make')['Electric_Range'].mean().reset_index()).sort_values(by='Electric_Range',ascending=False).reset_index(drop=True).head(10)
km_range.columns = ['model','km_range']
ps.pie(data_frame=km_range, names='model', values='km_range',hover_name='km_range',title='Top 10 Model with KM range',hole=True)

**Question 12. What are the Year Wise Cars sales growth?**

In [ ]:
# Year Wise Cars
year_wise_cars = df.groupby('ModelYear')['VIN'].count().reset_index()
year_wise_cars.columns = ['year','num_cars']
fig = ps.line(year_wise_cars,x="year", y="num_cars", title='Year Wise Number of Cars',markers=True)
fig.show()

In [ ]:
year_wise_cars.sort_values(by='num_cars', ascending=False).head(10)

**Question 13. Plotting the lat and lon**

In [ ]:
df['lat'] = df['Vehicle_Location'].str.split(' ').str[1].str.split('(').str[1].astype(float)
df['lon'] = df['Vehicle_Location'].str.split(' ').str[2].str.split(')').str[0].astype(float)
df['lat'] = df['lat'].fillna(0)
df['lon'] = df['lon'].fillna(0)

In [ ]:
geometry = [Point(xy) for xy in zip(df['lat'], df['lon'])]
gdf = GeoDataFrame(df, geometry=geometry)
#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red',markersize=15);